In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

# Fish, Weight, and Species

The following is a model to estimate the weight of a fish based on it's measurements.

The data used to train and test the model in question is found in the data folder as a csv file.

## Data

In [ ]:
# Read data into a Dataframe
fish_data = pd.read_csv('./data/Fish.csv')

fish_data.info()

While the data has no null values, it could and does have weights of zero.

This should not be possible, and these entries will be removed in cleaning.

In [ ]:
# Drop Weights of 0
index = fish_data[ fish_data['Weight'] == 0.0].index
fish_data.drop(index, inplace = True)

fish_data.describe()

In this model, the species of each fish in the dataset was not used.  The model was found to be more prone to error and less accurate while taking this portion into account.

In [ ]:
# Separate Target from features
y_fish = fish_data.iloc[:, 1]
X_fish = fish_data.drop(columns=['Weight'])
X_fish = X_fish.drop(columns=['Species'])

# Convert into numpy arrays
X_fish = X_fish.values
y_fish = y_fish.values

# Split into Training and Test Groups
X_fish_train, X_fish_test, y_fish_train, y_fish_test = train_test_split(X_fish, y_fish, test_size = 0.2, random_state = 0)

## Training

The model is built using sci-kit's Ridge model with an alpha of 0.

While this would technically be linear regression without utilizing the normalization that comes with the Ridge model, the Linear Regression model in sci-kit produced worse results.

In [ ]:
# Build Model
regressor = make_pipeline(PolynomialFeatures(3), Ridge(alpha=0.0))

# Train on split data
regressor.fit(X_fish_train, y_fish_train)

## Evaluation

The model will be evaluated utilizing three methods:

1) Comparing Predictions to Actual Test Values
2) Mean Squared Error
3) R2 Score

The last three will be done using the results of the first evaluation method.

In [ ]:
# Predict using Test Set
y_fish_pol_pred = regressor.predict(X_fish_test)

# Build a comparison Dataframe with predictions and actual
pred_pol_compare = pd.DataFrame()
pred_pol_compare['Prediction'] = y_fish_pol_pred.tolist()
pred_pol_compare['Actual'] = y_fish_test.tolist()

print(pred_pol_compare)

In [ ]:
# Evaluate using MSE and R2Score
mse_pol = mean_squared_error(y_fish_test, y_fish_pol_pred)
r2s_pol = r2_score(y_fish_test, y_fish_pol_pred)

print("MSE: %s | R2S: %s" % (mse_pol, r2s_pol))

## Attempted Models

These are some of the working models evaluated throughout the process to find the presented model above.

A variety of methods and hyperparameters were tested in each case.  A short description of each approach will be pinned at the top of each section.

Each model uses the same data as described in the 'Data' section above in the csv file.

### Multiple Linear Regression

The baseline attempt to estimate fish weights uses a simple multiple linear regression model.

Sci-kit's Linear Regression is used with multiple features as described in the data set.

Achieves a R2 Score of about 0.82.

In [ ]:
# Build Model
regressor = LinearRegression()

# Train on split data
regressor.fit(X_fish_train, y_fish_train)

In [ ]:
# Predict using Test Set
y_fish_pol_pred = regressor.predict(X_fish_test)

# Build a comparison Dataframe with predictions and actual
pred_pol_compare = pd.DataFrame()
pred_pol_compare['Prediction'] = y_fish_pol_pred.tolist()
pred_pol_compare['Actual'] = y_fish_test.tolist()

print(pred_pol_compare)

In [ ]:
# Evaluate using MSE and R2Score
mse_pol = mean_squared_error(y_fish_test, y_fish_pol_pred)
r2s_pol = r2_score(y_fish_test, y_fish_pol_pred)

print("MSE: %s | R2S: %s" % (mse_pol, r2s_pol))

### Polynomial Multiple Linear Regression

A polynomial multiple linear regression that serves as the baseline for the presented model.

Uses a polynomial transform of degree 2 on the data trained on Sci-kit's Linear Regression model.

Achieves a R2 Score of about 0.93.

In [ ]:
# Build Model
regressor = make_pipeline(PolynomialFeatures(2), LinearRegression())

# Train on split data
regressor.fit(X_fish_train, y_fish_train)

In [ ]:
# Predict using Test Set
y_fish_pol_pred = regressor.predict(X_fish_test)

# Build a comparison Dataframe with predictions and actual
pred_pol_compare = pd.DataFrame()
pred_pol_compare['Prediction'] = y_fish_pol_pred.tolist()
pred_pol_compare['Actual'] = y_fish_test.tolist()

print(pred_pol_compare)

In [ ]:
# Evaluate using MSE and R2Score
mse_pol = mean_squared_error(y_fish_test, y_fish_pol_pred)
r2s_pol = r2_score(y_fish_test, y_fish_pol_pred)

print("MSE: %s | R2S: %s" % (mse_pol, r2s_pol))

### Species as Categorical Data

A multiple linear regression model that takes into account the species of each entry.

The species are one-hot encoded in the following cells to remove value from enumeration.  Other than the addition of these features, the model is the same as the baseline multiple linear regression described above.

Achieves a R2 Score of about 0.88.

In [ ]:
# Create a copy of fish_data
fish_data_species = fish_data

# Enumerate every species to a number value
level_dict = {level:i for i,level in enumerate(fish_data['Species'].unique())}

# Use encode function to receive species value
def integer_encode_level(level):
    return level_dict[level]

# Apply the function to the Species column
fish_data_species['Species'] = fish_data_species['Species'].apply(integer_encode_level)

# One-hot-encode this feature
fish_data_species = pd.concat([fish_data_species, pd.get_dummies(fish_data_species['Species'], prefix = 'Species')], axis = 1)
fish_data_species.drop('Species', axis = 1, inplace = True)

# Check data
fish_data_species

In [ ]:
# Separate Target from features
y_fish_categorical = fish_data_species.iloc[:, 0]
X_fish_categorical = fish_data_species.drop(columns=['Weight'])

# Convert into numpy arrays
X_fish_categorical = X_fish_categorical.values
y_fish_categorical = y_fish_categorical.values

# Split into Training and Test Groups
X_fish_train_categorical, X_fish_test_categorical, y_fish_train_categorical, y_fish_test_categorical = train_test_split(X_fish_categorical, y_fish_categorical, test_size = 0.2, random_state = 0)

In [ ]:
# Build Model
regressor = LinearRegression()

# Train on split data
regressor.fit(X_fish_train_categorical, y_fish_train_categorical)

In [ ]:
# Predict using Test Set
y_fish_pred_categorical = regressor.predict(X_fish_test_categorical)

# Build a comparison Dataframe with predictions and actual
pred_cat_compare = pd.DataFrame()
pred_cat_compare['Prediction'] = y_fish_pred_categorical.tolist()
pred_cat_compare['Actual'] = y_fish_test_categorical.tolist()

print(pred_cat_compare)

In [ ]:
# Evaluate using MSE and R2Score
mse_cat = mean_squared_error(y_fish_test_categorical, y_fish_pred_categorical)
r2s_cat = r2_score(y_fish_test_categorical, y_fish_pred_categorical)

print("MSE: %s | R2S: %s" % (mse_cat, r2s_cat))

### Polynomial Multiple Linear Regression with Categorical Data

A polynomial multiple linear regression model that takes into account categorical data.

The categorical data is the same set processed in the previous model, this time using a more optimized and tested model with a pipeline and the Ridge model from Sci-kit's library.

Achieves a R2 Score of about 0.95.

In [ ]:
# Build Model
regressor = make_pipeline(PolynomialFeatures(3), Ridge(alpha=60.0))

# Train on split data
regressor.fit(X_fish_train_categorical, y_fish_train_categorical)

In [ ]:
# Predict using Test Set
y_fish_pol_cat_pred = regressor.predict(X_fish_test_categorical)

# Build a comparison Dataframe with predictions and actual
pred_pol_compare = pd.DataFrame()
pred_pol_compare['Prediction'] = y_fish_pol_cat_pred.tolist()
pred_pol_compare['Actual'] = y_fish_test_categorical.tolist()

print(pred_pol_compare)

In [ ]:
# Evaluate using MSE and R2Score
mse_pol = mean_squared_error(y_fish_test_categorical, y_fish_pol_cat_pred)
r2s_pol = r2_score(y_fish_test_categorical, y_fish_pol_cat_pred)

print("MSE: %s | R2S: %s" % (mse_pol, r2s_pol))